# Human Annotated NOTAM Analysis

Please note that the data for this analysis is saved locally in a `MatchingLaunchNOTAM_GMUCapstone_20220603` folder that is one level above (a parent folder) to where the [GitHub](https://github.com/DAEN-Team-1/Final "GitHub") files have been saved. This was done because a lot of the data files are too large to be included in the GitHub repoo.

In [1]:
# imports
import pandas as pd
# import bamboolib as bam

Read in data

In [2]:
# Read in csv (not enough headers for data, so using custom one)
names = ['LAUNCHES_REC_ID', 'NOTAM_REC_ID', 'LowerAltitude', 'UpperAltitude', 'IssueTime', 'StartTime', 'StopTime', 'E_CODE',
         '0_no_header', '1_no_header', '2_no_header', '3_no_header', '4_no_header', '5_no_header', '6_no_header', '7_no_header',
         '8_no_header', '9_no_header', '10_no_header', '11_no_header', '12_no_header', '13_no_header', '14_no_header']

df = pd.read_csv('../MatchingLaunchNOTAM_GMUCapstone_20220603/HumanAnnotatedMatches_SVO_DB_20200127.csv',
                 encoding='UTF-8', on_bad_lines='skip', engine="python", delimiter=',',
                 names=names)                

In [4]:
# Drop previous headers
df = df.drop(0)
df.reset_index(inplace=True, drop=True)

In [6]:
df.head()

,LAUNCHES_REC_ID,NOTAM_REC_ID,LowerAltitude,UpperAltitude,IssueTime,StartTime,StopTime,E_CODE,0_no_header,1_no_header,...,5_no_header,6_no_header,7_no_header,8_no_header,9_no_header,10_no_header,11_no_header,12_no_header,13_no_header,14_no_header
0,391,848687,0,50000,2018-04-01 11:24:00,2018-04-02 18:22:00,2018-04-02 21:08:00,WARNING AREA W497A ACT,None,None,...,None,None,None,None,None,None,None,None,None,None
1,391,835580,0,180000,2018-03-28 09:47:00,2018-04-02 19:52:00,2018-04-02 21:08:00,FL..AIRSPACE CAPE CANAVERAL FL..TEMPORARY FLT ...,FLT LIMITATION IN THE PROXIMITY OF SPACE FLT...,OPS BY FAA CERT PILOTS OR U.S.REG ACFT ARE PR...,...,R2934,AND PORTIONS OF W137F,W137G,W497A. ADDITIONAL WARNING AND RESTRICTED AREA...,None,None,None,None,None,None
2,391,835582,0,180000,2018-03-28 09:48:00,2018-04-02 19:52:00,2018-04-02 21:08:00,FL..AIRSPACE CAPE CANAVERAL FL..TEMPORARY FLT ...,FLT LIMITATION IN THE PROXIMITY OF SPACE FLT...,OPS BY FAA CERT PILOTS OR U.S.REG ACFT ARE PR...,...,R2934,AND PORTIONS OF W137F,W137G,W497A. ADDITIONAL WARNING AND RESTRICTED AREA...,None,None,None,None,None,None
3,391,848685,0,180000,2018-03-28 09:48:00,2018-04-02 19:52:00,2018-04-02 21:08:00,CAPE CANAVERAL FL..TEMPORARY FLT RESTRICTION....,FLT LIMITATION IN THE PROXIMITY OF SPACE FLT...,OPS BY FAA CERT PILOTS OR U.S.REG ACFT ARE PR...,...,R2934,AND PORTIONS OF W137F,W137G,W497A. ADDITIONAL WARNING AND RESTRICTED AREA...,None,None,None,None,None,None
4,391,848684,0,180000,2018-03-28 09:50:00,2018-04-02 19:52:00,2018-04-02 21:08:00,TEMPORARY FLT RESTRICTION. PURSUANT TO 14 CFR...,FLT LIMITATION IN THE PROXIMITY OF SPACE FLT...,OPS BY FAA CERT PILOTS OR U.S.REG ACFT ARE PR...,...,R2934,AND PORTIONS OF W137F,W137G,W497A. ADDITIONAL WARNING AND RESTRICTED AREA...,None,None,None,None,None,None


In [8]:
# Look at longest row
df.iloc[141]

LAUNCHES_REC_ID                                                  263
NOTAM_REC_ID                                                   82263
LowerAltitude                                                      0
UpperAltitude                                                 180000
IssueTime                                        2016-08-16 09:15:00
StartTime                                        2016-08-19 02:47:00
StopTime                                         2016-08-19 06:23:00
E_CODE                      EROP D6556 ATTENTION AIRLINE DISPATCHERS
0_no_header         SOME OPERATORS APPEAR TO BE USING THE NOTAM B...
1_no_header         KZNY ARTCC/OAC &amp; TJZS ARTCC /OAC WILL NOT...
2_no_header                             50NM BETWEEN 6000W AND 5500W
3_no_header         AND 60NM EAST OF 5500W OR WITHIN AFFECTED WAR...
4_no_header        R2934 AND CFR 91.143 SFC-UNL. EFF: 1608190447 ...
5_no_header                                                     AR15
6_no_header                       